<a href="https://colab.research.google.com/github/adheeb4/7PAM2015-0105-2023_LLM/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train'][0])

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:

import re


# Function to clean the text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text) # Remove text inside parentheses
    text = re.sub('"','', text) # Remove double quotes
    text = re.sub("'",'', text) # Remove single quotes
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    return text

# Clean the dataset
def clean_dataset(example):
    example['article'] = clean_text(example['article'])
    example['highlights'] = clean_text(example['highlights'])
    return example

dataset = dataset.map(clean_dataset)

# Print the first example to verify
print(dataset['train'][0])


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

{'article': 'london, england -- harry potter star daniel radcliffe gains access to a reported £20 million fortune as he turns 18 on monday, but he insists the money wont cast a spell on him. daniel radcliffe as harry potter in harry potter and the order of the phoenix to the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. i dont plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar, he told an australian interviewer earlier this month. i dont think ill be particularly extravagant. the things i like buying are things that cost about 10 pounds -- books and cds and dvds. at 18, radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film hostel: part ii, currently six places below his number one movie on the uk box office chart. details of how hell mark his landmark bi

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
import os

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# Tokenize and pad sequences
def tokenize_and_pad_sequences(texts, max_length, output_file):
    num_batches = len(texts) // 64 + (len(texts) % 64 != 0)

    with open(output_file, 'w') as f:
        f.write('input_ids,attention_mask,labels\n')
        for i in range(num_batches):
            batch = texts[i * 64 : (i + 1) * 64]
            inputs = tokenizer(batch, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")
            for j in range(len(batch)):
                f.write(','.join([str(inputs['input_ids'][j].numpy().tolist()),
                                  str(inputs['attention_mask'][j].numpy().tolist()),
                                  str(inputs['input_ids'][j].numpy().tolist())]) + '\n')

# Tokenize and pad articles
train_articles = dataset['train']['article']
train_summaries = dataset['train']['highlights']

val_articles = dataset['validation']['article']
val_summaries = dataset['validation']['highlights']

test_articles = dataset['test']['article']
test_summaries = dataset['test']['highlights']

max_length = 512 # Define maximum sequence length

# Save the preprocessed data
preprocessed_data_path = "preprocessed_cnn_dataset"
if not os.path.exists(preprocessed_data_path):
    os.makedirs(preprocessed_data_path)

tokenize_and_pad_sequences(train_articles, max_length, os.path.join(preprocessed_data_path, 'train.csv'))
tokenize_and_pad_sequences(val_articles, max_length, os.path.join(preprocessed_data_path, 'val.csv'))
tokenize_and_pad_sequences(test_articles, max_length, os.path.join(preprocessed_data_path, 'test.csv'))

print("Preprocessing completed and data saved.")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Preprocessing completed and data saved.


In [ ]:
import pandas as pd
import tensorflow as tf

# Load preprocessed data
preprocessed_data_path = "preprocessed_cnn_dataset"

train_data = pd.read_csv(os.path.join(preprocessed_data_path, 'train.csv'))
val_data = pd.read_csv(os.path.join(preprocessed_data_path, 'val.csv'))
test_data = pd.read_csv(os.path.join(preprocessed_data_path, 'test.csv'))

# Convert data to TensorFlow Dataset
train_dataset = tf.data.experimental.make_csv_dataset(
    os.path.join(preprocessed_data_path, 'train.csv'),
    batch_size=8,
    num_epochs=1,
    column_defaults=[tf.string, tf.string, tf.string],
    select_columns=['input_ids', 'attention_mask', 'labels']
)

val_dataset = tf.data.experimental.make_csv_dataset(
    os.path.join(preprocessed_data_path, 'val.csv'),
    batch_size=8,
    num_epochs=1,
    column_defaults=[tf.string, tf.string, tf.string],
    select_columns=['input_ids', 'attention_mask', 'labels']
)

test_dataset = tf.data.experimental.make_csv_dataset(
    os.path.join(preprocessed_data_path, 'test.csv'),
    batch_size=8,
    num_epochs=1,
    column_defaults=[tf.string, tf.string, tf.string],
    select_columns=['input_ids', 'attention_mask', 'labels']
)

# Convert string columns to integer
def parse_data(row):
    input_ids = tf.strings.to_number(tf.strings.split(row['input_ids'], ','))
    attention_mask = tf.strings.to_number(tf.strings.split(row['attention_mask'], ','))
    labels = tf.strings.to_number(tf.strings.split(row['labels'], ','))
    return {'input_ids': input_ids, 'attention_mask': attention_mask}, {'labels': labels}

train_dataset = train_dataset.map(parse_data)
val_dataset = val_dataset.map(parse_data)
test_dataset = test_dataset.map(parse_data)

print("Datasets configured for training.")


Datasets configured for training.


In [ ]:
print(test_data)

In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
pip install accelerate -U

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, TFAutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments

# Load pre-trained model
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_steps=1000,
    save_steps=1000,
    output_dir="./",
    overwrite_output_dir=True,
    num_train_epochs=3,
    warmup_steps=500,
    save_total_limit=1,
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(eval_results)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']


In [ ]:
print(dataset)


In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# Tokenize and pad sequences
def tokenize_and_pad_sequences(texts, max_length):
    inputs = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")
    return inputs

# Function to preprocess and save data
def preprocess_and_save_data(data_split, max_length, batch_size, output_file):
    inputs_list = []
    outputs_list = []

    for i in range(0, len(data_split), batch_size):
        texts = data_split[i:i+batch_size]
        inputs = tokenize_and_pad_sequences(texts, max_length)
        inputs_list.append(inputs)

    inputs = {key: tf.concat([inp[key] for inp in inputs_list], axis=0) for key in inputs_list[0].keys()}

    for i in range(0, len(data_split), batch_size):
        texts = data_split[i:i+batch_size]
        outputs = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")
        outputs_list.append(outputs)

    outputs = {key: tf.concat([out[key] for out in outputs_list], axis=0) for key in outputs_list[0].keys()}

    data = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': outputs['input_ids']
    }

    pd.DataFrame(data).to_csv(output_file, index=False)

# Tokenize and pad articles
train_articles = dataset['train']['article']
train_summaries = dataset['train']['highlights']

val_articles = dataset['validation']['article']
val_summaries = dataset['validation']['highlights']

test_articles = dataset['test']['article']
test_summaries = dataset['test']['highlights']

max_length = 512 # Define maximum sequence length
batch_size = 64 # Define batch size for processing

# Save the preprocessed data
preprocessed_data_path = "preprocessed_cnn_dataset"
if not os.path.exists(preprocessed_data_path):
    os.makedirs(preprocessed_data_path)

preprocess_and_save_data(train_articles, max_length, batch_size, os.path.join(preprocessed_data_path, 'train.csv'))
preprocess_and_save_data(val_articles, max_length, batch_size, os.path.join(preprocessed_data_path, 'val.csv'))
preprocess_and_save_data(test_articles, max_length, batch_size, os.path.join(preprocessed_data_path, 'test.csv'))

print("Preprocessing completed and data saved.")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# Tokenize and pad sequences
def tokenize_and_pad_sequences(texts, max_length):
    inputs = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")
    return inputs

# Tokenize and pad articles
train_articles = dataset['train']['article']
train_summaries = dataset['train']['highlights']

val_articles = dataset['validation']['article']
val_summaries = dataset['validation']['highlights']

test_articles = dataset['test']['article']
test_summaries = dataset['test']['highlights']

max_length = 512 # Define maximum sequence length

train_inputs = tokenize_and_pad_sequences(train_articles, max_length)
train_outputs = tokenizer(train_summaries, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")

val_inputs = tokenize_and_pad_sequences(val_articles, max_length)
val_outputs = tokenizer(val_summaries, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")

test_inputs = tokenize_and_pad_sequences(test_articles, max_length)
test_outputs = tokenizer(test_summaries, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")

# Save the preprocessed data
preprocessed_data_path = "preprocessed_cnn_dataset"
if not os.path.exists(preprocessed_data_path):
    os.makedirs(preprocessed_data_path)

train_data = {
    'input_ids': train_inputs['input_ids'],
    'attention_mask': train_inputs['attention_mask'],
    'labels': train_outputs['input_ids']
}

val_data = {
    'input_ids': val_inputs['input_ids'],
    'attention_mask': val_inputs['attention_mask'],
    'labels': val_outputs['input_ids']
}

test_data = {
    'input_ids': test_inputs['input_ids'],
    'attention_mask': test_inputs['attention_mask'],
    'labels': test_outputs['input_ids']
}

pd.DataFrame(train_data)
#.to_csv(os.path.join(preprocessed_data_path, 'train.csv'), index=False)
pd.DataFrame(val_data)
#.to_csv(os.path.join(preprocessed_data_path, 'val.csv'), index=False)
pd.DataFrame(test_data)
#.to_csv(os.path.join(preprocessed_data_path, 'test.csv'), index=False)

print("Preprocessing completed and data saved.")


In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Get the lengths of articles and summaries
article_lengths = [len(article.split()) for article in dataset['train']['article']]
summary_lengths = [len(summary.split()) for summary in dataset['train']['highlights']]

# Plot the distributions
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(article_lengths, bins=50, color='blue', alpha=0.7)
plt.title('Article Length Distribution')
plt.xlabel('Length')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
plt.hist(summary_lengths, bins=30, color='green', alpha=0.7)
plt.title('Summary Length Distribution')
plt.xlabel('Length')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Print statistics
print("Article Length:")
print("Min:", min(article_lengths))
print("Max:", max(article_lengths))
print("Mean:", sum(article_lengths) / len(article_lengths))

print("\nSummary Length:")
print("Min:", min(summary_lengths))
print("Max:", max(summary_lengths))
print("Mean:", sum(summary_lengths) / len(summary_lengths))
